In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

file_path = 'dataset_comments.xlsx'

df = pd.read_excel(file_path, names=['UserSenderId', 'SubmitDate', 'MessageText'])

def clean_html(content):
    if isinstance(content, str):
        # Use BeautifulSoup to remove HTML tags
        soup = BeautifulSoup(content, 'html.parser')
        return soup.get_text()
    return content  # Return as is if not a string

df['Text'] = df['MessageText'].apply(clean_html)

In [ ]:
df.head()

,UserSenderId,SubmitDate,MessageText,Text
0,14771,2023-05-16 19:27:21.773,"<p class=""st0"">В общем podman ...",В общем podman - это не наш метод. На нем раб...
1,8194,2023-05-16 16:56:35.380,"<p class=""st0"">А зачем ввод ИНН? Если его нет ...","А зачем ввод ИНН? Если его нет у компании, то ..."
2,18355,2023-05-16 15:08:17.097,сориентируйте по размерам пожалуйста,сориентируйте по размерам пожалуйста
3,18551,2023-05-16 09:33:51.050,"толстовка хорошая, но начес лезет сильно. над...","толстовка хорошая, но начес лезет сильно. над..."
4,14634,2023-05-15 14:21:37.207,"<p class=""st0"">Здорово! Нащупали идеальный фор...",Здорово! Нащупали идеальный формат 😎👍👍Из предл...


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="cointegrated/rubert-tiny-sentiment-balanced", batch_size=8, truncation=True)

Device set to use cuda:0


In [ ]:
res = pipe(df['Text'].to_list())

In [ ]:
df['label'] = [el['label'] for el in res]
df['score'] = [el['score'] for el in res]

In [ ]:
df.to_excel('a.xlsx', index=False)

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

model_checkpoint = "cointegrated/rubert-tiny-sentiment-balanced"
save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/vocab.txt',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')

In [ ]:
from optimum.pipelines import pipeline
classifier = pipeline(task="text-classification", accelerator="ort", model = "cointegrated/rubert-tiny-sentiment-balanced", device='cpu')

Device set to use cpu


In [ ]:
classifier(df['Text'][3])

[{'label': 'positive', 'score': 0.706735372543335}]